In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
#frametime is wrong, don't use it (20fps instead of 25)

In [2]:
A = np.genfromtxt('./cleaned_gps/6/6_lat_lon_rel.csv', delimiter=',')
camera_stamps = pd.read_csv('./cleaned_gps/6/SOURCE_GPS_LOG_6_cleaned.csv', delimiter=',')
gps_stamps = pd.read_csv('./cleaned_gps/6/gps_time_6.csv', delimiter=',')
fps = 25

In [14]:
camera_stamps.loc[camera_stamps.estimated_time != '00:00:00']['Frame No.'].iloc[0]
#camera_stamps.iloc[16]

16

In [179]:
for i in range(16):
    camera_stamps.estimated_time.iloc[i] = '19:33:58'
for i, stamp in enumerate(camera_stamps.estimated_time):
    offset = 9/25+i/25
    camera_stamps.estimated_time[i] = stamp + str(round(offset - int(offset),2))[1:]
camera_stamps.to_csv('./cleaned_gps/6/corrected_camera_loc.csv', index = False)

D:\Conda\envs\tensorflow\lib\site-packages\pandas\core\indexing.py:671: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)
D:\Conda\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


In [180]:
frequencies = dict(gps_stamps.timestamp.value_counts())
for stamp, count in frequencies.items():
    for i,(index,val) in enumerate(zip(gps_stamps[gps_stamps.timestamp == stamp].index, gps_stamps[gps_stamps.timestamp == stamp].timestamp)):
        offset = (i+1)/25*int(25/(count+1))
        gps_stamps.timestamp.iloc[index] = stamp + str(offset - int(offset))[1:]
gps_stamps['lat'] = A[:,0]
gps_stamps['lon'] = A[:,1]
gps_stamps.to_csv('./cleaned_gps/6/gps_time_6_interp.csv', index = False)

In [183]:
camera_stamps.head()

,Frame No.,UTC Timestamp,Lat,Long,estimated_time
0,0,19:33:54,32.70297,-117.234631,19:33:58.36
1,1,19:33:54,32.70297,-117.234631,19:33:58.4
2,2,19:33:54,32.70297,-117.234631,19:33:58.44
3,3,19:33:54,32.70297,-117.234631,19:33:58.48
4,4,19:33:54,32.70297,-117.234631,19:33:58.52


In [184]:
camera_stamps[camera_stamps.estimated_time ==  '19:34:02.48']

,Frame No.,UTC Timestamp,Lat,Long,estimated_time
103,103,19:33:59,32.70297,-117.234631,19:34:02.48


In [209]:
lat = []
lon = []
for i,stamp in enumerate(camera_stamps.estimated_time):
    if str(stamp) in gps_stamps.timestamp.values:
        lat.append(gps_stamps[gps_stamps.timestamp == stamp].iloc[0].lat)
        lon.append(gps_stamps[gps_stamps.timestamp == stamp].iloc[0].lon)
    else:
        lat.append(None)
        lon.append(None)

In [212]:
camera_stamps['boat_lat_rel'] = lat
camera_stamps['boat_lon_rel'] = lon

In [217]:
ground_truth = camera_stamps.drop(['Lat','Long'],axis=1)

In [218]:
ground_truth.to_csv('correlated_ground_truth_6.csv', index = False)